In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
finance = pd.read_csv('data/reportfinance/TCB_reportfinance.csv', sep='\t')
finance.info()
finance.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 54 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CAN DOI KE TOAN  75 non-null     object 
 1    2021            47 non-null     float64
 2    2020            47 non-null     float64
 3    2020.1          47 non-null     float64
 4    2020.2          47 non-null     float64
 5    2020.3          47 non-null     float64
 6    2020.4          47 non-null     float64
 7    2019            47 non-null     float64
 8    2019.1          47 non-null     float64
 9    2019.2          47 non-null     float64
 10   2019.3          47 non-null     float64
 11   2019.4          47 non-null     float64
 12   2018            47 non-null     float64
 13   2018.1          47 non-null     float64
 14   2018.2          47 non-null     float64
 15   2018.3          47 non-null     float64
 16   2018.4          47 non-null     float64
 17   2017            4

,CAN DOI KE TOAN,2021,2020,2020.1,2020.2,2020.3,2020.4,2019,2019.1,2019.2,...,2012.2,2012.3,2011,2010,2009,2008,2007,2006,2005,2004
0,Tài sản ngắn hạn###Current Assets,376030633.0,341647851.0,341647851.0,291917300.0,329295693.0,301455162.0,333907287.0,333907287.0,296048093.0,...,118181565.0,139191864.0,149281784.0,146467360.0,94590051.0,57992153.0,39093501.0,20489521.0,12500834.0,9517251.0
1,Tiền và các khoản tương đương tiền###Cash and ...,38361058.0,28994954.0,28994954.0,26597264.0,45284968.0,30990383.0,47990224.0,47990224.0,43304052.0,...,28783220.0,38513264.0,43190766.0,46831156.0,26268954.0,15525959.0,9303685.0,4458308.0,2632576.0,3073580.0
2,Các khoản đầu tư tài chính ngắn hạn###Short te...,43870182.0,37342530.0,37342530.0,36652144.0,54625255.0,41427137.0,58031780.0,58031780.0,50282030.0,...,28942287.0,38964143.0,43528612.0,47319342.0,26740727.0,16447209.0,9948685.0,7335112.0,4575196.0,3073580.0
3,Các khoản phải thu ngắn hạn###Short term Accou...,0.0,0.0,0.0,0.0,229385470.0,229037642.0,227885283.0,227885283.0,202462011.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Hàng tồn kho###Inventory,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
def get_bs(finance):
    start_bs = 'Tài sản ngắn hạn###Current Assets'
    end_bs = 'TỔNG CỘNG NGUỒN VỐN ###TOTAL EQUITY'
    bs = (
        finance
        .rename(columns={' 2021' : '2021.4'})
        .set_index('CAN DOI KE TOAN')
        .loc[start_bs: end_bs]
        .reset_index()
    )
    return bs

In [74]:
def get_inc(finance):
    start_inc = 'Tổng doanh thu hoạt động kinh doanh###Gross Sale Revenues'
    end_inc = 'Lợi nhuận sau thuế thu nhập doanh nghiệp###Profit after Corporate Income Tax'
    inc = (
        finance
        .rename(columns={' 2021' : '2021.4'})
        .set_index('CAN DOI KE TOAN')
        .loc[start_inc: end_inc]
        .reset_index()
    )
    return inc

In [75]:
def get_eng_element(df):
    df = df.rename(columns = {'CAN DOI KE TOAN' : 'Elements'})
    element_split = df['Elements'].str.split('###')
    eng_element = []
    for x in element_split:
        if len(x) == 1:
            eng_element.append(x[0])
        if len(x) == 2:
            eng_element.append(x[1])

    eng_element = (
        pd.Series(eng_element)
        .replace('Cash and Cash Euivalents', 'Cash and Cash Equivalents')
        .replace('Tài sản cố định hữu hình - Giá trị hao mòn lũy kế', 'Tangible Assets')
        .replace('Tài sản cố định thuê tài chính - Giá trị hao mòn lũy kế', 'Leased Assets')
        .replace('Tài sản cố định vô hình - Giá trị hao mòn lũy kế', 'Intangible Assets')
        .replace('Lợi thế thương mại', 'Goodwill')
        .replace('Dự phòng nghiệp vụ', 'Provision')
        .replace('No khac', 'Other Liabilities')
        .replace('Lợi ích của cổ đông thiểu số', 'Minority Interest')
        .str.title()
        .str.replace('-', '_')
        .str.replace(' ', '_')
    )
    df['Elements'] = eng_element
    return df

In [76]:
def _get_yr_qtr_cols(df):
    cols = df.columns.str.strip()
    yr_cols = pd.Series(cols)[lambda s: s.map(lambda x: len(x)==4)]
    qtr_cols = pd.Series(cols)[lambda s: s.map(lambda x: len(x)==6)]
    return yr_cols, qtr_cols

In [77]:
def get_qtr_cols(df):
    df.columns = df.columns.str.strip()
    yr_cols, qtr_cols = _get_yr_qtr_cols(df)
    df_qtr = df[['Elements'] + qtr_cols.to_list()]
    return df_qtr

In [78]:
def get_yr_cols(df):
    df.columns = df.columns.str.strip()
    yr_cols, qtr_cols = _get_yr_qtr_cols(df)
    df_yr = df[['Elements'] + yr_cols.to_list()]
    return df_yr

In [79]:
def _convert_suffix_to_mth(suffix):
    if str(suffix) == '1':
        mth = '12'
    if str(suffix) == '2':
        mth = '09'
    if str(suffix) == '3':
        mth = '06'
    if str(suffix) == '4':
        mth = '03'
    return mth

In [80]:
def _add_mths(mths, no_mths=1, fmt='%Y%m'):
    mths = pd.to_datetime(mths, format=fmt) + timedelta(days=30 * (no_mths + 0.5))
    mths = mths.dt.to_period('M').dt.strftime(fmt)
    return mths

In [81]:
def change_suffix_to_mths(bs_qtr):
    cols_suffix = pd.Series(bs_qtr.columns[1:])
    yrs = cols_suffix.str[0:4]
    mths = cols_suffix.str[5].apply(_convert_suffix_to_mth)
    cols_months = _add_mths(yrs + mths, 1)
    bs_qtr.columns = ['Elements'] + list(cols_months)
    return bs_qtr

In [82]:
finance.columns

Index(['CAN DOI KE TOAN', ' 2021', ' 2020', ' 2020.1', ' 2020.2', ' 2020.3',
       ' 2020.4', ' 2019', ' 2019.1', ' 2019.2', ' 2019.3', ' 2019.4', ' 2018',
       ' 2018.1', ' 2018.2', ' 2018.3', ' 2018.4', ' 2017', ' 2017.1',
       ' 2017.2', ' 2017.3', ' 2017.4', ' 2016', ' 2016.1', ' 2016.2',
       ' 2016.3', ' 2016.4', ' 2015', ' 2015.1', ' 2015.2', ' 2015.3',
       ' 2015.4', ' 2014', ' 2014.1', ' 2014.2', ' 2014.3', ' 2014.4', ' 2013',
       ' 2013.1', ' 2013.2', ' 2013.3', ' 2013.4', ' 2012', ' 2012.1',
       ' 2012.2', ' 2012.3', ' 2011', ' 2010', ' 2009', ' 2008', ' 2007',
       ' 2006', ' 2005', ' 2004'],
      dtype='object')

In [88]:
bs = get_bs(finance)
bs_eng = get_eng_element(bs)
bs_qtr = get_qtr_cols(bs_eng)
bs_mth = change_suffix_to_mths(bs_qtr)

In [106]:
(
    bs_mth
    .set_index('Elements')
    .transpose()
    .reset_index()
    .rename(columns={'index': 'Feat_Month'})
    .assign(Ticker = 'TCB')
    .loc[lambda df: df['Feat_Month'].between('202007', '202104')]
    .drop('Feat_Month', axis=1)
    .groupby('Ticker')
    .mean()
    # .columns
)

Elements,Current_Assets,Cash_And_Cash_Equivalents,Short_Term_Financial_Investment,Short_Term_Account_Receivables,Inventory,Other_Current_Assets,Non_Current_Assets,Long_Term_Account_Receivable,Fixed_Assets,Tangible_Assets,...,Liabilities,Short_Term_Liabilities,Long_Term_Liabilities,Provision,Other_Liabilities,Owners_Equity,Expenditures_And_Other_Funds,Owners_Equity,Minority_Interest,Total_Equity
Ticker,,,,,,,,,,,,,,,,,,,,,
TCB,3.347229e+08,34809561.0,43122527.75,57346367.5,0.0,0.0,NaN,0.0,4434571.0,-210226.5,...,352000824.0,73449896.75,8658065.25,0.0,8263882.75,72936276.75,0.0,16763419.75,93881.0,4.249371e+08


In [16]:
bs = get_bs(finance)
bs_eng = get_eng_element(bs)
bs_yr = get_yr_cols(bs_eng)
bs_yr

,Elements,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004
0,Current_Assets,376030633.0,341647851.0,333907287.0,236245058.0,226070456.0,192342897.0,142578057.0,119278956.0,100849782.0,130546045.0,149281784.0,146467360.0,94590051.0,57992153.0,39093501.0,20489521.0,12500834.0,9517251.0
1,Cash_And_Cash_Equivalents,38361058.0,28994954.0,47990224.0,35559363.0,30155807.0,21598874.0,14762552.0,18922460.0,15420747.0,31299956.0,43190766.0,46831156.0,26268954.0,15525959.0,9303685.0,4458308.0,2632576.0,3073580.0
2,Short_Term_Financial_Investment,43870182.0,37342530.0,58031780.0,43131592.0,36950193.0,29623494.0,17354140.0,21008706.0,16340355.0,32109782.0,43528612.0,47319342.0,26740727.0,16447209.0,9948685.0,7335112.0,4575196.0,3073580.0
3,Short_Term_Account_Receivables,0.0,0.0,227885283.0,157554103.0,158964456.0,141120529.0,110461365.0,79347790.0,69088680.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Inventory,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Other_Current_Assets,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Non_Current_Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Long_Term_Account_Receivable,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Fixed_Assets,4726319.0,4613423.0,3207777.0,1718596.0,1511446.0,1582722.0,882081.0,1036505.0,1032737.0,1146424.0,1191224.0,1003907.0,700901.0,2055026.0,1078723.0,651136.0,297585.0,144482.0
9,Tangible_Assets,0.0,0.0,-967984.0,-1047256.0,-1033167.0,-941451.0,-803851.0,-704367.0,-577781.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
inc = get_inc(finance)
inc_eng = get_eng_element(inc)
inc_qtr = get_qtr_cols(inc_eng)
inc_mth = change_suffix_to_mths(inc_qtr)
inc_mth

,Elements,202101,202010,202007,202004,202001,201910,201907,201904,201901,...,201410,201407,201404,201401,201310,201307,201304,201301,201210,201207
0,Gross_Sale_Revenues,7678930.0,7700905.0,6630891.0,7004186.0,6628302.0,6272798.0,5864332.0,6002834.0,5576449.0,...,3353513.0,3086728.0,3174903.0,3108179.0,3168474.0,3223030.0,3781622.0,3675170.0,4217842.0,19948573.0
1,Deduction_Revenues,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Net_Sales,0.0,0.0,3935378.0,4212412.0,3904209.0,3622066.0,3124025.0,3359469.0,2959012.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Cost_Of_Goods_Sold,2184997.0,2553225.0,2695513.0,2791774.0,2724093.0,2650732.0,2740307.0,2643365.0,2617437.0,...,1749410.0,1802858.0,1881120.0,2001297.0,2150263.0,2302487.0,2491596.0,2723929.0,2804128.0,14650198.0
4,Gross_Profit,5493933.0,5147680.0,3935378.0,4212412.0,3904209.0,3622066.0,3124025.0,3359469.0,2959012.0,...,1604103.0,1283870.0,1293783.0,1106882.0,1018211.0,920543.0,1290026.0,951241.0,1413714.0,5298375.0
5,Financial_Activities_Revenues,0.0,0.0,1817989.0,1818052.0,2722968.0,1736525.0,1563724.0,1035159.0,2096353.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Financial_Expenses,0.0,0.0,1269814.0,1172031.0,1490285.0,978914.0,965478.0,990402.0,1351481.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Selling_Expenses,0.0,0.0,1523122.0,1309379.0,1861972.0,1204502.0,1177837.0,888172.0,1750460.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Managing_Expenses,0.0,0.0,1697493.0,2137714.0,2336823.0,1794313.0,1571063.0,1610310.0,2108849.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Net_Profit_From_Operating_Activities,5455123.0,5007584.0,4055874.0,3892750.0,4290354.0,3564278.0,3116791.0,2784318.0,2946516.0,...,1310730.0,768593.0,751813.0,559529.0,410047.0,608356.0,714238.0,-87972.0,758015.0,4562977.0


In [34]:
inc = get_inc(finance)
inc_eng = get_eng_element(inc)
inc_yr = get_yr_cols(inc_eng)
inc_yr

,Elements,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004
0,Gross_Sale_Revenues,8151158.0,29001912.0,25016341.0,21150222.0,17594504.0,15736077.0,13374087.0,12931617.0,13281305.0,17622864.0,10934383.0,6882366.0,6213718.0,2326002.0,1207503.0,790227.0,442263.0,NaN
1,Deduction_Revenues,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,Net_Sales,0.0,0.0,14257844.0,11126535.0,8930412.0,8142221.0,7208380.0,5772630.0,4335662.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,Cost_Of_Goods_Sold,2027437.0,10250703.0,10758497.0,10023687.0,8664092.0,7593856.0,6165707.0,7158987.0,8945643.0,12507291.0,7750034.0,4382546.0,4469416.0,1400728.0,750056.0,438961.0,264929.0,NaN
4,Gross_Profit,6123721.0,18751209.0,14257844.0,11126535.0,8930412.0,8142221.0,7208380.0,5772630.0,4335662.0,5115573.0,3184349.0,2499820.0,1744302.0,925274.0,457447.0,351266.0,177334.0,NaN
5,Financial_Activities_Revenues,0.0,0.0,6810301.0,7223233.0,7413394.0,3776505.0,2135562.0,1333802.0,1312174.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
6,Financial_Expenses,0.0,0.0,4425079.0,3147169.0,1963425.0,1653250.0,1511730.0,985922.0,738646.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
7,Selling_Expenses,0.0,0.0,4884408.0,4451721.0,4519685.0,2558990.0,1682123.0,1665405.0,1150038.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
8,Managing_Expenses,0.0,0.0,7312509.0,5842507.0,4698283.0,4260995.0,3682803.0,3431045.0,3355666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
9,Net_Profit_From_Operating_Activities,6369119.0,18411331.0,13755636.0,12507261.0,11645523.0,7657731.0,5661139.0,3675387.0,2292170.0,2467337.0,3131272.0,2734382.0,2364278.0,790627.0,387179.0,280250.0,130318.0,NaN
